# Simulation session
- Theory on Monte Carlo ray-tracing simulations
- Introduction to the McStas package
- Learn basics of the Python API McStasScript

## Monte Carlo

In [ ]:
import monte_carlo
%matplotlib widget
monte_carlo.example_interactive(square_side_length=1.0)

## Monte Carlo ray-tracing

In [ ]:
monte_carlo.example_interactive(square_side_length=1.1)

### Random starting position and direction

In [ ]:
import simple_simulator
sim = simple_simulator.Simulator()
src = sim.add_component("Source", width=0.1, height=0.1, angle_spread=30)
sim.add_component("Propagator", distance=0.1)

rays = sim.run(num_rays=15)
sim.visualize(rays, show_coordinates=False)

### Example with guide

In [ ]:
import ray_tracer_examples
sim = ray_tracer_examples.guide()

rays = sim.run(num_rays=20)
sim.visualize(rays)

### Complex example

In [ ]:
import ray_tracer_examples
sim = ray_tracer_examples.large()

rays = sim.run(num_rays=100)
sim.visualize(rays)

# Monte Carlo ray-tracing in Neutron Scattering
- Used to simulate the facility
    - Target
    - Moderator
    - Guides
    - Choppers
    - Sample
    - Detectors
    - ...
- Predict performance
- Optimize source / instruments

# McStas
McStas was originally created in Denmark and is celebrating its 25th anniversary this year!

- Allows user to simulate an instrument through sequence of components place in space
- Has large component library
- Relatively easy to create new components

### Technical
- C programming language and code generation
- C-meta language for instrument file that describes simulation

### Coordiante system
- x: left, looking in the beam direction
- y: upwards, against gravity
- z: beam direction 

In [ ]:
import simple_simulator
sim = simple_simulator.Simulator()
src = sim.add_component("Source", width=0.1, height=0.1, angle_spread=3)
sim.add_component("Propagator", distance=0.1)

rays = sim.run(num_rays=5)
sim.visualize(rays, show_coordinates=0.06)

In [ ]:
sim = simple_simulator.Simulator()

src = sim.add_component("Source", width=0.05, height=0.05, angle_spread=3)
guide = sim.add_component("Guide", width=0.07, height=0.07, length=2,
                          position=[0,0,0.75], rotation=[0,0,0])
monitor = sim.add_component("Monitor", nx=20, ny=20, width=0.08, height=0.08,
                            position=[0,0,guide.length + 0.2], relative=guide)

rays = sim.run(num_rays=5)
sim.visualize(rays, show_coordinates=[0.06, 0.06, 0.25])

In [ ]:
import ray_tracer_examples
sim = ray_tracer_examples.large()
rays = sim.run(num_rays=20)
sim.visualize(rays, show_coordinates=[0.06, 0.06, 0.1])

### Units in McStas
Components can be contributed and could use any unit. Commonly SI + meV and Å.

Positions and rotation of components always in meters and degrees.

## McStasScript Python API
There exists a Python API to run McStas, here we will go through how to make a simple simulation.

### Import the McStasScript package

In [ ]:
import mcstasscript as ms

### Create instrument object

In [ ]:
instrument = ms.McStas_instr("python_tutorial")

### See available components

In [ ]:
instrument.available_components()

In [ ]:
instrument.available_components("sources")

### Get help for any component

In [ ]:
instrument.component_help("Source_div")

### Add a component to the instrument

In [ ]:
src = instrument.add_component("source", "Source_div")
instrument.show_components()

In [ ]:
print(src)

### Set component parameters

In [ ]:
src.set_parameters(xwidth=0.1, yheight=0.05,
                   focus_aw=1.2, focus_ah=2.3)
print(src)

### See all parameters

In [ ]:
src.show_parameters()

### Add instrument parameters

In [ ]:
wavelength_par = instrument.add_parameter("wavelength", value=5.0,
                                      comment="Wavelength in [Ang]")

wavelength_spread_par = instrument.add_parameter("wavelength_spread", value=1.2,
                                      comment="Wavelength spread in [Ang]")

In [ ]:
src.set_parameters(lambda0=wavelength_par, dlambda="wavelength_spread")
print(src)

### Place components in space

In [ ]:
guide_A = instrument.add_component("guide_A", "Guide_gravity")
guide_A.set_parameters(w1=0.05, h1=0.05, l=5, m=2.5)
guide_A.set_AT([0, 0, 2], RELATIVE="source")

In [ ]:
guide_B = instrument.add_component("guide_B", "Guide_gravity")
guide_B.set_parameters(w1=0.05, h1=0.05, l=5, m=2.5)
guide_B.set_AT([0, 0, guide_A.l + 0.2], RELATIVE=guide_A)

### Place a monitor component

In [ ]:
mon = instrument.add_component("monitor", "PSD_monitor")
mon.set_parameters(nx=100, ny=100, filename='"PSD.dat"',
                   xwidth=0.05, yheight=0.05, restore_neutron=1)
mon.set_AT([0, 0, guide_B.l + 0.5], RELATIVE=guide_B)

### See geometry

In [ ]:
instrument.show_instrument()

### Get overview of component sequence

In [ ]:
instrument.show_components()

### Place a component earlier in the sequence

In [ ]:
mon = instrument.add_component("monitor_in_guide", "PSD_monitor", after="guide_A")
mon.set_parameters(nx=100, ny=100, filename='"PSD_in_guide.dat"',
                   xwidth=0.05, yheight=0.05, restore_neutron=1)
mon.set_AT([0, 0, guide_A.l + 0.1], RELATIVE=guide_A)

### See diagram of instrument sequence

In [ ]:
instrument.show_diagram()

## Executing the simulation
### See what parameters are available

In [ ]:
instrument.show_parameters()

### Set parameters

In [ ]:
instrument.set_parameters(wavelength=2.8)
instrument.show_parameters()

### McStas settings

In [ ]:
instrument.settings(ncount=5E6, output_path="mcstas_data_set", suppress_output=True)
instrument.show_settings()

### Execute the simulation

In [ ]:
data = instrument.backengine()

### Plot the data

In [ ]:
ms.make_sub_plot(data, figsize=(10, 4))

### Run with different parameters

In [ ]:
instrument.set_parameters(wavelength=0.8, wavelength_spread=0.1)
instrument.settings(ncount=1E8, mpi=6)
data = instrument.backengine()

In [ ]:
ms.make_sub_plot(data, figsize=(10, 4))

# Exercises

Two choices:
- Elastic: Small Angle Neutron Scattering SANS
- Inelastic: Quasi-Elastic Neutron Scattering QENS

In both you will get an instrument and change it slightly.

There will be quiz questions to guide you with feedback.

In the coming days you will continue working with the data you generate today!

In [1]:
import quizlib
quiz = quizlib.SANS_Quiz()

In [ ]:
quiz.question_1()